In [47]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd
import csv
#import psycopg2

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = ["ps_trace-2018*"]

my_query = {}
my_query2 = {}
# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"
start_date = '20180406T000000Z'
end_date = '20180407T002000Z'

src_lists = ['144.92.180.76', '150.244.246.86']
src_to_dest = {}
for x in range (0,len(src_lists)):
    my_query = {
        "size":1,
        "_source": {
            "include": [ 'src','dest','hops','n_hops','timestamp' ]
        },
        'query':{
            'bool':{
                'must':[
                    {'range': {'timestamp': {'gte': start_date, 'lt': end_date}}},
#                    {'term': {'_type': 'traceroute'}},
                             {'bool':
                                 {'should':[
                                     {'term': {'src': src_lists[x]}},
                                     #{'term': {'dest': '150.244.246.86'}},
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
                                 ]}
                             }
                        #,
                             #{'bool':
                                #{'should':[
                                  #  {'term': {'dest': '150.244.246.86'}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
                            # ]}
                         #}
                ]

            }
        },
    }
results = elasticsearch.helpers.scan(es, query=my_query, index=my_index, request_timeout=100000)
for i in results:
    print(i['_source'])

{'timestamp': 1522972989000, 'hops': ['150.244.246.1', '193.145.14.25', '130.206.212.105', '130.206.245.2', '62.40.124.192', '62.40.98.73', '62.40.98.152', '198.71.45.236', '192.5.89.221', '192.5.89.58', '18.122.64.2', '192.5.207.252'], 'dest': '192.5.207.252', 'n_hops': 12, 'src': '150.244.246.86'}
{'timestamp': 1522973157000, 'hops': ['150.244.246.1', '130.206.206.157', '62.40.126.197', '62.40.126.186', '62.40.126.222', '193.51.191.77', '134.158.123.184'], 'dest': '134.158.123.184', 'n_hops': 7, 'src': '150.244.246.86'}
{'timestamp': 1523059801000, 'hops': ['150.244.246.1', '130.206.206.157', '62.40.126.197', '62.40.126.186', '62.40.126.222', '193.51.191.77', '134.158.123.184'], 'dest': '134.158.123.184', 'n_hops': 7, 'src': '150.244.246.86'}
{'timestamp': 1522976599000, 'hops': ['150.244.246.1', '130.206.206.157', '62.40.126.197', '62.40.126.186', '62.40.126.222', '193.51.191.77', '134.158.123.184'], 'dest': '134.158.123.184', 'n_hops': 7, 'src': '150.244.246.86'}
{'timestamp': 1522

In [29]:
    data_size = len(results['aggregations']['grouped_by_hash']['buckets'])

    for i in range(0, data_size):
        rt_src = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src']
        rt_src_host = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['src_host']
        rt_dest = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest']
        rt_dest_host = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['dest_host']
        rt_hops = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['hops']
        rt_ts = results['aggregations']['grouped_by_hash']['buckets'][i]['top_hash_hits']['hits']['hits'][0]['_source']['timestamp']
        print(rt_ts)
        dupe_rt = 0
        if rt_src in src_to_dest.keys():
            if rt_dest in src_to_dest[rt_src].keys():
                current_rt = 'rt1'
                for x in range(0, len(src_to_dest[rt_src][rt_dest])):
                    if src_to_dest[rt_src][rt_dest][current_rt]['hop_list'] == rt_hops:
                        src_to_dest[rt_src][rt_dest][current_rt]['count'] += 1
                        dupe_rt = 1
                    current_rt = 'rt' + str(x+2)
                if (dupe_rt == 0):
                    src_to_dest[rt_src][rt_dest][current_rt] = {}
                    src_to_dest[rt_src][rt_dest][current_rt]['count'] = 1
                    src_to_dest[rt_src][rt_dest][current_rt]['hop_list'] = rt_hops
            else:
                src_to_dest[rt_src][rt_dest] = {'rt1':{}}
                src_to_dest[rt_src][rt_dest]['rt1']['count'] = 1
                src_to_dest[rt_src][rt_dest]['rt1']['hop_list'] = rt_hops
            dupe_rt = 0
    #completely new source
        else:
            src_to_dest[rt_src] = {rt_dest:{'rt1':{}}}
            src_to_dest[rt_src][rt_dest]['rt1']['count'] = 1
            src_to_dest[rt_src][rt_dest]['rt1']['hop_list'] = rt_hops   
print(results)
print(len(results['aggregations']['grouped_by_hash']['buckets']))

KeyError: 'stored_fields'

In [55]:
diffrts = 0

for src_key in src_to_dest:
    for dest_key in src_to_dest[src_key]:
        for rt_num in src_to_dest[src_key][dest_key]:
            diffrts += 1
                
print(diffrts)

print(src_to_dest)

392
{'132.206.245.253': {'150.244.246.86': {'rt1': {'hop_list': ['132.206.245.254', '206.12.9.101', '205.189.32.77', '62.40.126.197', '62.40.126.198', '130.206.206.158', '150.244.246.86'], 'count': 2}}}, '206.12.154.59': {'150.244.246.86': {'rt1': {'hop_list': ['206.12.154.254', '142.104.252.153', '142.104.252.246', '207.23.241.113', '205.189.32.196', '205.189.32.199', '205.189.32.175', '205.189.32.177', '205.189.32.181', '62.40.124.221', '62.40.98.153', '62.40.98.72', '62.40.124.193', '130.206.245.1', '130.206.212.106', '193.145.14.26', '150.244.246.86'], 'count': 2}, 'rt2': {'hop_list': ['206.12.154.254', '142.104.252.153', '142.104.252.246', '207.23.241.113', '205.189.32.196', '205.189.32.199', '205.189.32.175', '205.189.32.177', '205.189.32.181', '62.40.124.221', '62.40.98.153', '62.40.98.72', '62.40.124.193', '130.206.212.106', '193.145.14.26', '150.244.246.86'], 'count': 2}}}, '193.109.172.187': {'150.244.246.86': {'rt1': {'hop_list': ['193.109.172.161', '130.206.206.153', '130.2